In [ ]:
import numpy as np
import torch
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from albumentations.pytorch.transforms import ToTensorV2

In [ ]:
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

WEIGHTS_FILE = "../input/fasterrcnn-weights/fasterrcnn_weights.bin"

DETECTION_THRESHOLD = 0.66

In [ ]:
def get_model():
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False, pretrained_backbone=False)
    num_classes = 2  
    # we set this to be 2: the starfish is one class and the second class is that of the background

    # we need to determine the number of input features
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    #load the trained weights
    model.load_state_dict(torch.load(WEIGHTS_FILE, map_location=DEVICE))
    model.eval()

    model = model.to(DEVICE)
    return model

model = get_model()

In [ ]:
def format_prediction_string(boxes, scores):
    prediction_strings = []
    for j in zip(scores, boxes):
        prediction_strings.append("{0:.10f} {1} {2} {3} {4}".format(j[0], j[1][0], j[1][1], j[1][2], j[1][3]))
    return " ".join(prediction_strings)


def predict(model, pixel_array):
    # Predictions for a single image
    pixel_array = pixel_array.astype(np.float32) / 255.
    tensor_img = ToTensorV2(p=1.0)(image=pixel_array)['image'].unsqueeze(0)
    
    # Get predictions
    with torch.no_grad():
        outputs = model(tensor_img.to(DEVICE))[0]

    boxes = outputs['boxes'].data.cpu().numpy()
    scores = outputs['scores'].data.cpu().numpy()
    
    boxes = boxes[scores >= DETECTION_THRESHOLD].astype(np.int32)
    scores = scores[scores >= DETECTION_THRESHOLD]

    boxes[:, 2] = boxes[:, 2] - boxes[:, 0]
    boxes[:, 3] = boxes[:, 3] - boxes[:, 1]
  
    # the evaluation tab specified the format for submission, hence we seek to return with the same format as requested
    return format_prediction_string(boxes, scores)

In [ ]:
import greatbarrierreef

env = greatbarrierreef.make_env()

iter_test = env.iter_test() 

for (pixel_array, df_pred) in iter_test:
    df_pred['annotations'] = predict(model, pixel_array)
    env.predict(df_pred)